In [1]:
import sys

import tensorflow as tf
from tensorflow import keras as kr

from time import sleep
import numpy as np
import random
import pandas as pd
pd.set_option('display.max_rows', 200)

import sklearn as sk

from sklearn.preprocessing import MinMaxScaler

import matplotlib 

import matplotlib.pyplot as plt

import zipfile
import os

import supportFunctions as sf

import inspect
lines = inspect.getsource(sf.highest)

In [2]:
#função utilizada para normalizar os dados de treino e validação
def normalize_data(train, test):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(train)
    train_scaled = scaler.transform(train)
    test_scaled = scaler.transform(test)
    
    return scaler, train_scaled, test_scaled

#função utilizada para desnormalizar os dados, no formato de entrada
def denormalize_data(scaler,data):
    result = scaler.inverse_transform(data)
    return result

#função utilizada para desnomalizar o valor predito
def denormalize_prediction(scaler, dataX, dataY):
    formatted_data = np.array(dataX)
    formatted_data[:,3] = np.array(dataY)
    
    return denormalize_data(scaler,formatted_data)[:,3]

In [3]:
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")
# tf.debugging.set_log_device_placement(False)

Tensor Flow Version: 2.12.0
Keras Version: 2.12.0

Python 3.9.16 (main, Jan 11 2023, 16:16:36) [MSC v.1916 64 bit (AMD64)]
Pandas 1.5.2
Scikit-Learn 1.2.1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is NOT AVAILABLE


In [4]:
#quantos passos para trás os indicadores técnicos vão olhar
ti_memory = 10

#lendo os dados que serão utilizados para treinamento e validação
ge1day = pd.read_csv('Data\GE.csv')
#removendo a coluna que indica a data
ge1day = ge1day.drop(['Date'], axis = 1)

df = ge1day
#reseting the index
df.index = range(len(df))

In [7]:
#adicionando os parametros de indicadores tecnicos ao dataframe
df["""highest_{}""".format(ti_memory)] = sf.highest(df.Close,ti_memory)
df["""lowest_{}""".format(ti_memory)] = sf.lowest(df.Close,ti_memory)
df["""wma_{}""".format(ti_memory)] = sf.wma(df.Close,ti_memory)
df["""ema_{}""".format(ti_memory)] = sf.ema(df.Close,ti_memory)
df["""hma_{}""".format(ti_memory)] = sf.hma(df.Close,ti_memory)
df["""macd_12_26"""] = sf.macd(df.Close,12,26)
df["""rsi_{}""".format(ti_memory)] = sf.rsi(df.Close,ti_memory)
df["""dpo_{}""".format(ti_memory)] = sf.dpo(df.Close,ti_memory)

#parametros nao utilizados:
# df["""sma_{}""".format(ti_memory)] = sf.sma(df.Close,ti_memory)
# df["""so_k_5"""] = sf.so_k(df.Close)
# df["""so_d_3"""] = sf.so_d(df.Close)
# df["""obv""".format(ti_memory)] = sf.obv(df.Close,df.Volume)

C:\Users\guilh\PythonCodes\supportFunctions.py:263: RuntimeWarning: divide by zero encountered in divide
  rs = ag_vector/al_vector #numpy arrays so therefore can use element wise division
C:\Users\guilh\PythonCodes\supportFunctions.py:263: RuntimeWarning: invalid value encountered in divide
  rs = ag_vector/al_vector #numpy arrays so therefore can use element wise division


In [8]:
display(df)

,Open,High,Low,Close,Adj Close,Volume,highest_10,lowest_10,wma_10,ema_10,hma_10,macd_12_26,rsi_10,dpo_10
0,0.751202,0.763722,0.743690,0.748698,0.001782,2156500,0.748698,0.748698,0.748698,0.748698,0.748698,0.000000,NaN,0.000000
1,0.744942,0.744942,0.738682,0.741186,0.001764,1477600,0.748698,0.741186,0.743690,0.744942,0.745359,0.000000,100.000000,-0.744942
2,0.741186,0.747446,0.726162,0.732422,0.001743,1837000,0.748698,0.732422,0.738056,0.740769,0.741708,0.000000,100.000000,-0.740769
3,0.732422,0.733674,0.701122,0.713642,0.001698,2725600,0.748698,0.713642,0.728290,0.733987,0.734112,0.000000,100.000000,-0.733987
4,0.713642,0.713642,0.691106,0.712390,0.001695,3095000,0.748698,0.712390,0.722990,0.729667,0.727268,0.000000,100.000000,-0.729667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14658,7.630000,8.300000,7.510000,8.120000,8.120000,123180900,8.120000,6.110000,7.062182,7.424700,7.239323,-1.116727,50.108961,-0.399000
14659,7.680000,7.870000,7.540000,7.620000,7.620000,93299000,8.120000,6.110000,7.175091,7.460209,7.697586,-1.029444,40.027642,-0.496000
14660,7.540000,7.940000,7.350000,7.890000,7.890000,86850200,8.120000,6.110000,7.341273,7.538353,8.019525,-0.927790,41.907705,-0.579000
14661,7.870000,8.180000,7.820000,7.940000,7.940000,121149900,8.120000,6.110000,7.494182,7.611380,8.165303,-0.833585,32.316173,-1.075000


In [9]:
#jogando alguns dados fora para termos todas variaveis incializadas
inicialization_steps = 30
prediction_ahead = 10

dataValues = df.values

trainSize = int(len(dataValues)*0.8)
testSize = len(dataValues) - trainSize

train = df.head(trainSize)
# train,minDataTrain,minMaxDataTrain = sf.normalizaTrain(train)
test = df.tail(testSize)
# test = sf.normalizaTest(test, minDataTrain, minMaxDataTrain)

scaler, train_normalized, test_normalized = normalize_data(train, test)

#deixando um valor de fora, para podermos prever o próximo valor quando for o último:
trainX = train_normalized[inicialization_steps:(len(train)-prediction_ahead)]
testX = test_normalized[0:(len(test)-prediction_ahead)]

#normalizando as entradas
# scalerX, trainX, testX = normalize_data(trainX, testX)

#pegando apenas o valor que queremos prever
trainY = train_normalized[(inicialization_steps+prediction_ahead):len(train),3]
testY = test_normalized[prediction_ahead:len(test),3]

#calculando a diferenca percentual do preco de fechamento entre um dia e outro
# trainY = (train.values[(inicialization_steps+1):len(train),3]/trainX[:,3])-1
# testY = (test.values[1:len(test),3]/testX[:,3])-1


# scalerY, trainY, testY = normalize_data(trainY, testY)

In [10]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(11690, 14)
(11690,)
(2923, 14)
(2923,)


In [11]:
#checking if it makes sense
# print(trainInputX[:10,(rnn_memory-1),3])
# print(trainInputY[:10])

In [12]:
lstm_memory = 50

trainInputX, trainInputY = sf.prepareData(trainX, trainY, lstm_memory)
testInputX, testInputY = sf.prepareData(testX, testY, lstm_memory)
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,:,0:14]
testInputX = testInputX [:,:,0:14]
print(trainInputX.shape)

trained_models_lstm_50 = []
trained_models_lstm_50_history = []

for i in range(0,20):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 256
    
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
    
    trained_models_lstm_50.append(kr.Sequential())
    # lstm_model.add(kr.layers.BatchNormalization())
    trained_models_lstm_50[i].add(kr.layers.LSTM(14,return_sequences = True))
    trained_models_lstm_50[i].add(kr.layers.LSTM(8))
    # lstm_model.add(kr.layers.Simplelstm(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # lstm_model.add(kr.layers.BatchNormalization())
    trained_models_lstm_50[i].add(kr.layers.Dense(1))
    trained_models_lstm_50[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_lstm_50_history.append(trained_models_lstm_50[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY),  callbacks=[callback]))
    
    trained_models_lstm_50[i].save('C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_50_batch_256_earlystop_valloss_'+str(i))

(11640, 50, 14)
(11640,)
(2873, 50, 14)
(2873,)
11690
11640
11690
11640
(11640, 50, 14)
Epoch 1/2000
46/46 [==============================] - 4s 30ms/step - loss: 0.0444 - val_loss: 0.0032
Epoch 2/2000
46/46 [==============================] - 1s 19ms/step - loss: 0.0014 - val_loss: 0.0025
Epoch 3/2000
46/46 [==============================] - 1s 19ms/step - loss: 7.1545e-04 - val_loss: 0.0039
Epoch 4/2000
46/46 [==============================] - 1s 19ms/step - loss: 5.5066e-04 - val_loss: 0.0049
Epoch 5/2000
46/46 [==============================] - 1s 19ms/step - loss: 4.8493e-04 - val_loss: 0.0045
Epoch 6/2000
46/46 [==============================] - 1s 19ms/step - loss: 4.3998e-04 - val_loss: 0.0044
Epoch 7/2000
46/46 [==============================] - 1s 19ms/step - loss: 4.0440e-04 - val_loss: 0.0040
Epoch 8/2000
46/46 [==============================] - 1s 19ms/step - loss: 3.8526e-04 - val_loss: 0.0034
Epoch 9/2000
46/46 [==============================] - 1s 19ms/step - loss: 3.611

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_50_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_50_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
46/46 [==============================] - 4s 29ms/step - loss: 0.0045 - val_loss: 0.0050
Epoch 2/2000
46/46 [==============================] - 1s 18ms/step - loss: 7.5138e-04 - val_loss: 0.0020
Epoch 3/2000
46/46 [==============================] - 1s 18ms/step - loss: 4.4014e-04 - val_loss: 0.0015
Epoch 4/2000
46/46 [==============================] - 1s 18ms/step - loss: 3.8125e-04 - val_loss: 0.0013
Epoch 5/2000
46/46 [==============================] - 1s 19ms/step - loss: 3.5380e-04 - val_loss: 0.0014
Epoch 6/2000
46/46 [==============================] - 1s 19ms/step - loss: 3.4176e-04 - val_loss: 0.0011
Epoch 7/2000
46/46 [==============================] - 1s 18ms/step - loss: 3.3707e-04 - val_loss: 0.0018
Epoch 8/2000
46/46 [==============================] - 1s 19ms/step - loss: 3.2067e-04 - val_loss: 0.0012
Epoch 9/2000
46/46 [==============================] - 1s 18ms/step - loss: 3.1594e-04 - val_loss: 0.0016
Epoch 10/2000
46/46 [==============================] - 1s 1

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_50_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_50_batch_256_earlystop_valloss_18\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_50_batch_256_earlystop_valloss_18\assets


Epoch 1/2000
46/46 [==============================] - 4s 30ms/step - loss: 0.0331 - val_loss: 0.0139
Epoch 2/2000
46/46 [==============================] - 1s 19ms/step - loss: 0.0022 - val_loss: 0.0021
Epoch 3/2000
46/46 [==============================] - 1s 19ms/step - loss: 9.8652e-04 - val_loss: 0.0017
Epoch 4/2000
46/46 [==============================] - 1s 19ms/step - loss: 6.4471e-04 - val_loss: 0.0017
Epoch 5/2000
46/46 [==============================] - 1s 19ms/step - loss: 4.9876e-04 - val_loss: 0.0015
Epoch 6/2000
46/46 [==============================] - 1s 19ms/step - loss: 4.3010e-04 - val_loss: 0.0013
Epoch 7/2000
46/46 [==============================] - 1s 19ms/step - loss: 3.9346e-04 - val_loss: 0.0013
Epoch 8/2000
46/46 [==============================] - 1s 19ms/step - loss: 3.6859e-04 - val_loss: 0.0012
Epoch 9/2000
46/46 [==============================] - 1s 19ms/step - loss: 3.5205e-04 - val_loss: 0.0011
Epoch 10/2000
46/46 [==============================] - 1s 19ms/

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_50_batch_256_earlystop_valloss_19\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_50_batch_256_earlystop_valloss_19\assets


In [13]:
lstm_memory = 100

trainInputX, trainInputY = sf.prepareData(trainX, trainY, lstm_memory)
testInputX, testInputY = sf.prepareData(testX, testY, lstm_memory)
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,:,0:14]
testInputX = testInputX [:,:,0:14]
print(trainInputX.shape)

trained_models_lstm_100 = []
trained_models_lstm_100_history = []

for i in range(0,20):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 256
    
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
    
    trained_models_lstm_100.append(kr.Sequential())
    # lstm_model.add(kr.layers.BatchNormalization())
    trained_models_lstm_100[i].add(kr.layers.LSTM(14,return_sequences = True))
    trained_models_lstm_100[i].add(kr.layers.LSTM(8))
    # lstm_model.add(kr.layers.Simplelstm(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # lstm_model.add(kr.layers.BatchNormalization())
    trained_models_lstm_100[i].add(kr.layers.Dense(1))
    trained_models_lstm_100[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_lstm_100_history.append(trained_models_lstm_100[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY),  callbacks=[callback]))
    
    trained_models_lstm_100[i].save('C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_100_batch_256_earlystop_valloss_'+str(i))

(11590, 100, 14)
(11590,)
(2823, 100, 14)
(2823,)
11690
11590
11690
11590
(11590, 100, 14)
Epoch 1/2000
46/46 [==============================] - 5s 48ms/step - loss: 0.0458 - val_loss: 0.0030
Epoch 2/2000
46/46 [==============================] - 2s 38ms/step - loss: 0.0014 - val_loss: 0.0024
Epoch 3/2000
46/46 [==============================] - 2s 38ms/step - loss: 7.2536e-04 - val_loss: 0.0037
Epoch 4/2000
46/46 [==============================] - 2s 38ms/step - loss: 5.5586e-04 - val_loss: 0.0047
Epoch 5/2000
46/46 [==============================] - 2s 38ms/step - loss: 4.8949e-04 - val_loss: 0.0041
Epoch 6/2000
46/46 [==============================] - 2s 39ms/step - loss: 4.4252e-04 - val_loss: 0.0038
Epoch 7/2000
46/46 [==============================] - 2s 38ms/step - loss: 4.0888e-04 - val_loss: 0.0040
Epoch 8/2000
46/46 [==============================] - 2s 38ms/step - loss: 3.8215e-04 - val_loss: 0.0032
Epoch 9/2000
46/46 [==============================] - 2s 38ms/step - loss: 3.

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_100_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_100_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
46/46 [==============================] - 5s 48ms/step - loss: 0.0047 - val_loss: 0.0065
Epoch 2/2000
46/46 [==============================] - 2s 37ms/step - loss: 8.0603e-04 - val_loss: 0.0017
Epoch 3/2000
46/46 [==============================] - 2s 37ms/step - loss: 4.5520e-04 - val_loss: 0.0012
Epoch 4/2000
46/46 [==============================] - 2s 38ms/step - loss: 3.7698e-04 - val_loss: 0.0015
Epoch 5/2000
46/46 [==============================] - 2s 37ms/step - loss: 3.5830e-04 - val_loss: 0.0021
Epoch 6/2000
46/46 [==============================] - 2s 37ms/step - loss: 3.5081e-04 - val_loss: 0.0016
Epoch 7/2000
46/46 [==============================] - 2s 37ms/step - loss: 3.3861e-04 - val_loss: 0.0014
Epoch 8/2000
46/46 [==============================] - 2s 37ms/step - loss: 3.5176e-04 - val_loss: 0.0019
Epoch 9/2000
46/46 [==============================] - 2s 37ms/step - loss: 3.1692e-04 - val_loss: 0.0012
Epoch 10/2000
46/46 [==============================] - 2s 3

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_100_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_100_batch_256_earlystop_valloss_18\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_100_batch_256_earlystop_valloss_18\assets


Epoch 1/2000
46/46 [==============================] - 4s 38ms/step - loss: 0.0343 - val_loss: 0.0152
Epoch 2/2000
46/46 [==============================] - 1s 29ms/step - loss: 0.0022 - val_loss: 0.0017
Epoch 3/2000
46/46 [==============================] - 1s 30ms/step - loss: 0.0010 - val_loss: 0.0015
Epoch 4/2000
46/46 [==============================] - 1s 29ms/step - loss: 6.5617e-04 - val_loss: 0.0017
Epoch 5/2000
46/46 [==============================] - 1s 29ms/step - loss: 5.0779e-04 - val_loss: 0.0019
Epoch 6/2000
46/46 [==============================] - 1s 29ms/step - loss: 4.4327e-04 - val_loss: 0.0015
Epoch 7/2000
46/46 [==============================] - 1s 29ms/step - loss: 4.0204e-04 - val_loss: 0.0013
Epoch 8/2000
46/46 [==============================] - 1s 29ms/step - loss: 3.7695e-04 - val_loss: 0.0014
Epoch 9/2000
46/46 [==============================] - 1s 29ms/step - loss: 3.5481e-04 - val_loss: 0.0011
Epoch 10/2000
46/46 [==============================] - 1s 29ms/step

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_100_batch_256_earlystop_valloss_19\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_100_batch_256_earlystop_valloss_19\assets


In [14]:
lstm_memory = 200

trainInputX, trainInputY = sf.prepareData(trainX, trainY, lstm_memory)
testInputX, testInputY = sf.prepareData(testX, testY, lstm_memory)
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,:,0:14]
testInputX = testInputX [:,:,0:14]
print(trainInputX.shape)

trained_models_lstm_200 = []
trained_models_lstm_200_history = []

for i in range(0,20):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 256
    
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
    
    trained_models_lstm_200.append(kr.Sequential())
    # lstm_model.add(kr.layers.BatchNormalization())
    trained_models_lstm_200[i].add(kr.layers.LSTM(14,return_sequences = True))
    trained_models_lstm_200[i].add(kr.layers.LSTM(8))
    # lstm_model.add(kr.layers.Simplelstm(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # lstm_model.add(kr.layers.BatchNormalization())
    trained_models_lstm_200[i].add(kr.layers.Dense(1))
    trained_models_lstm_200[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_lstm_200_history.append(trained_models_lstm_200[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY),  callbacks=[callback]))
    
    trained_models_lstm_200[i].save('C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_200_batch_256_earlystop_valloss_'+str(i))

(11490, 200, 14)
(11490,)
(2723, 200, 14)
(2723,)
11690
11490
11690
11490
(11490, 200, 14)
Epoch 1/2000
45/45 [==============================] - 6s 83ms/step - loss: 0.0482 - val_loss: 0.0028
Epoch 2/2000
45/45 [==============================] - 3s 72ms/step - loss: 0.0014 - val_loss: 0.0020
Epoch 3/2000
45/45 [==============================] - 3s 72ms/step - loss: 7.3434e-04 - val_loss: 0.0032
Epoch 4/2000
45/45 [==============================] - 3s 73ms/step - loss: 5.6191e-04 - val_loss: 0.0037
Epoch 5/2000
45/45 [==============================] - 3s 72ms/step - loss: 4.9494e-04 - val_loss: 0.0041
Epoch 6/2000
45/45 [==============================] - 3s 73ms/step - loss: 4.5062e-04 - val_loss: 0.0038
Epoch 7/2000
45/45 [==============================] - 3s 73ms/step - loss: 4.1532e-04 - val_loss: 0.0035
Epoch 8/2000
45/45 [==============================] - 3s 73ms/step - loss: 3.9278e-04 - val_loss: 0.0030
Epoch 9/2000
45/45 [==============================] - 3s 72ms/step - loss: 3.

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_200_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_200_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
45/45 [==============================] - 6s 84ms/step - loss: 0.0047 - val_loss: 0.0056
Epoch 2/2000
45/45 [==============================] - 3s 73ms/step - loss: 7.6563e-04 - val_loss: 0.0020
Epoch 3/2000
45/45 [==============================] - 3s 73ms/step - loss: 4.4216e-04 - val_loss: 0.0015
Epoch 4/2000
45/45 [==============================] - 3s 73ms/step - loss: 3.9245e-04 - val_loss: 0.0018
Epoch 5/2000
45/45 [==============================] - 3s 73ms/step - loss: 3.5675e-04 - val_loss: 0.0010
Epoch 6/2000
45/45 [==============================] - 3s 74ms/step - loss: 3.5842e-04 - val_loss: 0.0017
Epoch 7/2000
45/45 [==============================] - 3s 73ms/step - loss: 3.2828e-04 - val_loss: 0.0013
Epoch 8/2000
45/45 [==============================] - 3s 73ms/step - loss: 3.3301e-04 - val_loss: 0.0013
Epoch 9/2000
45/45 [==============================] - 3s 73ms/step - loss: 3.3468e-04 - val_loss: 0.0013
Epoch 10/2000
45/45 [==============================] - 3s 7

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_200_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_200_batch_256_earlystop_valloss_18\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_200_batch_256_earlystop_valloss_18\assets


Epoch 1/2000
45/45 [==============================] - 6s 84ms/step - loss: 0.0338 - val_loss: 0.0165
Epoch 2/2000
45/45 [==============================] - 3s 74ms/step - loss: 0.0023 - val_loss: 0.0016
Epoch 3/2000
45/45 [==============================] - 3s 74ms/step - loss: 0.0010 - val_loss: 0.0015
Epoch 4/2000
45/45 [==============================] - 3s 73ms/step - loss: 6.6759e-04 - val_loss: 0.0016
Epoch 5/2000
45/45 [==============================] - 3s 74ms/step - loss: 5.1515e-04 - val_loss: 0.0020
Epoch 6/2000
45/45 [==============================] - 3s 74ms/step - loss: 4.3805e-04 - val_loss: 0.0014
Epoch 7/2000
45/45 [==============================] - 3s 73ms/step - loss: 4.0009e-04 - val_loss: 0.0014
Epoch 8/2000
45/45 [==============================] - 3s 74ms/step - loss: 3.8264e-04 - val_loss: 0.0015
Epoch 9/2000
45/45 [==============================] - 3s 73ms/step - loss: 3.5782e-04 - val_loss: 0.0013
Epoch 10/2000
45/45 [==============================] - 3s 74ms/step

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_200_batch_256_earlystop_valloss_19\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_200_batch_256_earlystop_valloss_19\assets


In [15]:
lstm_memory = 400

trainInputX, trainInputY = sf.prepareData(trainX, trainY, lstm_memory)
testInputX, testInputY = sf.prepareData(testX, testY, lstm_memory)
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,:,0:14]
testInputX = testInputX [:,:,0:14]
print(trainInputX.shape)

trained_models_lstm_400 = []
trained_models_lstm_400_history = []

for i in range(0,10):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 256
    
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
    
    trained_models_lstm_400.append(kr.Sequential())
    # lstm_model.add(kr.layers.BatchNormalization())
    trained_models_lstm_400[i].add(kr.layers.LSTM(14,return_sequences = True))
    trained_models_lstm_400[i].add(kr.layers.LSTM(8))
    # lstm_model.add(kr.layers.Simplelstm(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # lstm_model.add(kr.layers.BatchNormalization())
    trained_models_lstm_400[i].add(kr.layers.Dense(1))
    trained_models_lstm_400[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_lstm_400_history.append(trained_models_lstm_400[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY),  callbacks=[callback]))
    
    trained_models_lstm_400[i].save('C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_400_batch_256_earlystop_valloss_'+str(i))

(11290, 400, 14)
(11290,)
(2523, 400, 14)
(2523,)
11690
11290
11690
11290
(11290, 400, 14)
Epoch 1/2000
45/45 [==============================] - 9s 154ms/step - loss: 0.0489 - val_loss: 0.0029
Epoch 2/2000
45/45 [==============================] - 6s 144ms/step - loss: 0.0015 - val_loss: 0.0018
Epoch 3/2000
45/45 [==============================] - 7s 145ms/step - loss: 7.4320e-04 - val_loss: 0.0030
Epoch 4/2000
45/45 [==============================] - 6s 143ms/step - loss: 5.6988e-04 - val_loss: 0.0037
Epoch 5/2000
45/45 [==============================] - 6s 144ms/step - loss: 5.0594e-04 - val_loss: 0.0038
Epoch 6/2000
45/45 [==============================] - 6s 144ms/step - loss: 4.5923e-04 - val_loss: 0.0036
Epoch 7/2000
45/45 [==============================] - 6s 144ms/step - loss: 4.3070e-04 - val_loss: 0.0036
Epoch 8/2000
45/45 [==============================] - 6s 144ms/step - loss: 3.9572e-04 - val_loss: 0.0033
Epoch 9/2000
45/45 [==============================] - 6s 144ms/step -

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_400_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_400_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
45/45 [==============================] - 9s 154ms/step - loss: 0.0048 - val_loss: 0.0064
Epoch 2/2000
45/45 [==============================] - 6s 144ms/step - loss: 8.2020e-04 - val_loss: 0.0012
Epoch 3/2000
45/45 [==============================] - 6s 143ms/step - loss: 4.6606e-04 - val_loss: 0.0022
Epoch 4/2000
45/45 [==============================] - 6s 143ms/step - loss: 4.2531e-04 - val_loss: 0.0019
Epoch 5/2000
45/45 [==============================] - 6s 143ms/step - loss: 3.7562e-04 - val_loss: 0.0016
Epoch 6/2000
45/45 [==============================] - 6s 143ms/step - loss: 3.4620e-04 - val_loss: 0.0012
Epoch 7/2000
45/45 [==============================] - 6s 143ms/step - loss: 3.4647e-04 - val_loss: 0.0014
Epoch 8/2000
45/45 [==============================] - 6s 143ms/step - loss: 3.3005e-04 - val_loss: 9.8481e-04
Epoch 9/2000
45/45 [==============================] - 6s 142ms/step - loss: 3.2514e-04 - val_loss: 0.0012
Epoch 10/2000
45/45 [=========================

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_400_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_400_batch_256_earlystop_valloss_8\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_400_batch_256_earlystop_valloss_8\assets


Epoch 1/2000
45/45 [==============================] - 10s 159ms/step - loss: 0.0213 - val_loss: 0.0026
Epoch 2/2000
45/45 [==============================] - 7s 145ms/step - loss: 0.0012 - val_loss: 0.0015
Epoch 3/2000
45/45 [==============================] - 6s 144ms/step - loss: 6.3740e-04 - val_loss: 6.7212e-04
Epoch 4/2000
45/45 [==============================] - 6s 144ms/step - loss: 5.0291e-04 - val_loss: 9.9304e-04
Epoch 5/2000
45/45 [==============================] - 7s 145ms/step - loss: 4.4155e-04 - val_loss: 8.1866e-04
Epoch 6/2000
45/45 [==============================] - 7s 145ms/step - loss: 3.8326e-04 - val_loss: 7.7306e-04
Epoch 7/2000
45/45 [==============================] - 6s 143ms/step - loss: 3.6553e-04 - val_loss: 9.6025e-04
Epoch 8/2000
45/45 [==============================] - 6s 144ms/step - loss: 3.4372e-04 - val_loss: 9.3865e-04
Epoch 9/2000
45/45 [==============================] - 6s 144ms/step - loss: 3.3887e-04 - val_loss: 9.4324e-04
Epoch 10/2000
45/45 [====

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_400_batch_256_earlystop_valloss_9\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_400_batch_256_earlystop_valloss_9\assets


In [16]:
lstm_memory = 800

trainInputX, trainInputY = sf.prepareData(trainX, trainY, lstm_memory)
testInputX, testInputY = sf.prepareData(testX, testY, lstm_memory)
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,:,0:14]
testInputX = testInputX [:,:,0:14]
print(trainInputX.shape)

trained_models_lstm_800 = []
trained_models_lstm_800_history = []

for i in range(0,5):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 256
    
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=24)
    
    trained_models_lstm_800.append(kr.Sequential())
    # lstm_model.add(kr.layers.BatchNormalization())
    trained_models_lstm_800[i].add(kr.layers.LSTM(14,return_sequences = True))
    trained_models_lstm_800[i].add(kr.layers.LSTM(8))
    # lstm_model.add(kr.layers.Simplelstm(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # lstm_model.add(kr.layers.BatchNormalization())
    trained_models_lstm_800[i].add(kr.layers.Dense(1))
    trained_models_lstm_800[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_lstm_800_history.append(trained_models_lstm_800[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY),  callbacks=[callback]))
    
    trained_models_lstm_800[i].save('C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_800_batch_256_earlystop_valloss_'+str(i))

(10890, 800, 14)
(10890,)
(2123, 800, 14)
(2123,)
11690
10890
11690
10890
(10890, 800, 14)
Epoch 1/2000
43/43 [==============================] - 15s 300ms/step - loss: 0.0505 - val_loss: 0.0029
Epoch 2/2000
43/43 [==============================] - 12s 287ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 3/2000
43/43 [==============================] - 12s 288ms/step - loss: 8.0492e-04 - val_loss: 0.0025
Epoch 4/2000
43/43 [==============================] - 12s 288ms/step - loss: 6.0391e-04 - val_loss: 0.0034
Epoch 5/2000
43/43 [==============================] - 12s 288ms/step - loss: 5.2872e-04 - val_loss: 0.0036
Epoch 6/2000
43/43 [==============================] - 12s 288ms/step - loss: 4.8343e-04 - val_loss: 0.0034
Epoch 7/2000
43/43 [==============================] - 12s 289ms/step - loss: 4.4935e-04 - val_loss: 0.0035
Epoch 8/2000
43/43 [==============================] - 12s 285ms/step - loss: 4.1843e-04 - val_loss: 0.0032
Epoch 9/2000
43/43 [==============================] - 12s 282

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_800_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_800_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
43/43 [==============================] - 16s 306ms/step - loss: 0.0048 - val_loss: 0.0062
Epoch 2/2000
43/43 [==============================] - 13s 306ms/step - loss: 8.2263e-04 - val_loss: 0.0021
Epoch 3/2000
43/43 [==============================] - 13s 308ms/step - loss: 4.7462e-04 - val_loss: 0.0018
Epoch 4/2000
43/43 [==============================] - 13s 310ms/step - loss: 4.0900e-04 - val_loss: 0.0015
Epoch 5/2000
43/43 [==============================] - 13s 309ms/step - loss: 3.8197e-04 - val_loss: 0.0017
Epoch 6/2000
43/43 [==============================] - 13s 301ms/step - loss: 3.6348e-04 - val_loss: 0.0017
Epoch 7/2000
43/43 [==============================] - 13s 300ms/step - loss: 3.4759e-04 - val_loss: 0.0011
Epoch 8/2000
43/43 [==============================] - 13s 310ms/step - loss: 3.4734e-04 - val_loss: 8.4056e-04
Epoch 9/2000
43/43 [==============================] - 13s 300ms/step - loss: 3.4893e-04 - val_loss: 9.1509e-04
Epoch 10/2000
43/43 [============

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_800_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_800_batch_256_earlystop_valloss_3\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_800_batch_256_earlystop_valloss_3\assets


Epoch 1/2000
43/43 [==============================] - 16s 310ms/step - loss: 0.0504 - val_loss: 0.0303
Epoch 2/2000
43/43 [==============================] - 13s 296ms/step - loss: 0.0021 - val_loss: 0.0154
Epoch 3/2000
43/43 [==============================] - 13s 297ms/step - loss: 0.0011 - val_loss: 0.0115
Epoch 4/2000
43/43 [==============================] - 13s 298ms/step - loss: 8.3213e-04 - val_loss: 0.0081
Epoch 5/2000
43/43 [==============================] - 13s 297ms/step - loss: 6.7955e-04 - val_loss: 0.0054
Epoch 6/2000
43/43 [==============================] - 13s 296ms/step - loss: 5.6456e-04 - val_loss: 0.0031
Epoch 7/2000
43/43 [==============================] - 13s 297ms/step - loss: 4.8635e-04 - val_loss: 0.0021
Epoch 8/2000
43/43 [==============================] - 13s 297ms/step - loss: 4.2777e-04 - val_loss: 0.0013
Epoch 9/2000
43/43 [==============================] - 13s 300ms/step - loss: 3.9126e-04 - val_loss: 0.0014
Epoch 10/2000
43/43 [============================

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_800_batch_256_earlystop_valloss_4\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_800_batch_256_earlystop_valloss_4\assets


In [17]:
lstm_memory = 1000

trainInputX, trainInputY = sf.prepareData(trainX, trainY, lstm_memory)
testInputX, testInputY = sf.prepareData(testX, testY, lstm_memory)
print(trainInputX.shape)
print(trainInputY.shape)
print(testInputX.shape)
print(testInputY.shape)

#validacao tamanho apos tratamentos
print(len(trainX))
print(len(trainInputX))

print(len(trainY))
print(len(trainInputY))

trainInputX = trainInputX [:,:,0:14]
testInputX = testInputX [:,:,0:14]
print(trainInputX.shape)

trained_models_lstm_1000 = []
trained_models_lstm_1000_history = []

for i in range(0,5):

    #giving it reproducibility
    seed = (i+1000)

    os.environ['PYTHONHASHseed']=str(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # new flag present in tf 2.0+
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    batch_size = 256
    
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=24)
    
    trained_models_lstm_1000.append(kr.Sequential())
    # lstm_model.add(kr.layers.BatchNormalization())
    trained_models_lstm_1000[i].add(kr.layers.LSTM(14,return_sequences = True))
    trained_models_lstm_1000[i].add(kr.layers.LSTM(8))
    # lstm_model.add(kr.layers.Simplelstm(5,input_shape=(trainInputX.shape[1],trainInputX.shape[2])))
    # lstm_model.add(kr.layers.BatchNormalization())
    trained_models_lstm_1000[i].add(kr.layers.Dense(1))
    trained_models_lstm_1000[i].compile(loss='mean_squared_error',optimizer='adam')
    
    trained_models_lstm_1000_history.append(trained_models_lstm_1000[i].fit(trainInputX, trainInputY, epochs=2000, batch_size=batch_size, verbose = 1, validation_data=(testInputX,testInputY),  callbacks=[callback]))
    
    trained_models_lstm_1000[i].save('C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_1000_batch_256_earlystop_valloss_'+str(i))

(10690, 1000, 14)
(10690,)
(1923, 1000, 14)
(1923,)
11690
10690
11690
10690
(10690, 1000, 14)
Epoch 1/2000
42/42 [==============================] - 22s 387ms/step - loss: 0.0540 - val_loss: 0.0035
Epoch 2/2000
42/42 [==============================] - 16s 377ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 3/2000
42/42 [==============================] - 16s 377ms/step - loss: 8.4357e-04 - val_loss: 0.0026
Epoch 4/2000
42/42 [==============================] - 16s 376ms/step - loss: 6.2885e-04 - val_loss: 0.0032
Epoch 5/2000
42/42 [==============================] - 16s 376ms/step - loss: 5.4645e-04 - val_loss: 0.0035
Epoch 6/2000
42/42 [==============================] - 15s 364ms/step - loss: 5.0031e-04 - val_loss: 0.0036
Epoch 7/2000
42/42 [==============================] - 13s 311ms/step - loss: 4.6256e-04 - val_loss: 0.0036
Epoch 8/2000
42/42 [==============================] - 13s 311ms/step - loss: 4.3312e-04 - val_loss: 0.0034
Epoch 9/2000
42/42 [==============================] - 13s 

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_1000_batch_256_earlystop_valloss_0\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_1000_batch_256_earlystop_valloss_0\assets


Epoch 1/2000
42/42 [==============================] - 15s 322ms/step - loss: 0.0049 - val_loss: 0.0063
Epoch 2/2000
42/42 [==============================] - 13s 311ms/step - loss: 8.7377e-04 - val_loss: 0.0022
Epoch 3/2000
42/42 [==============================] - 13s 310ms/step - loss: 4.9057e-04 - val_loss: 0.0015
Epoch 4/2000
42/42 [==============================] - 13s 311ms/step - loss: 4.4058e-04 - val_loss: 0.0018
Epoch 5/2000
42/42 [==============================] - 13s 311ms/step - loss: 3.8661e-04 - val_loss: 0.0018
Epoch 6/2000
42/42 [==============================] - 13s 311ms/step - loss: 3.6913e-04 - val_loss: 0.0012
Epoch 7/2000
42/42 [==============================] - 13s 311ms/step - loss: 3.6002e-04 - val_loss: 0.0017
Epoch 8/2000
42/42 [==============================] - 13s 310ms/step - loss: 3.6256e-04 - val_loss: 8.3160e-04
Epoch 9/2000
42/42 [==============================] - 13s 311ms/step - loss: 3.5815e-04 - val_loss: 0.0015
Epoch 10/2000
42/42 [================

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_1000_batch_256_earlystop_valloss_1\assets


...

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_1000_batch_256_earlystop_valloss_3\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_1000_batch_256_earlystop_valloss_3\assets


Epoch 1/2000
42/42 [==============================] - 16s 329ms/step - loss: 0.0528 - val_loss: 0.0298
Epoch 2/2000
42/42 [==============================] - 13s 320ms/step - loss: 0.0022 - val_loss: 0.0176
Epoch 3/2000
42/42 [==============================] - 14s 322ms/step - loss: 0.0012 - val_loss: 0.0126
Epoch 4/2000
42/42 [==============================] - 14s 323ms/step - loss: 8.6875e-04 - val_loss: 0.0082
Epoch 5/2000
42/42 [==============================] - 14s 323ms/step - loss: 6.9953e-04 - val_loss: 0.0058
Epoch 6/2000
42/42 [==============================] - 14s 322ms/step - loss: 5.8455e-04 - val_loss: 0.0037
Epoch 7/2000
42/42 [==============================] - 14s 323ms/step - loss: 4.9956e-04 - val_loss: 0.0020
Epoch 8/2000
42/42 [==============================] - 14s 323ms/step - loss: 4.3902e-04 - val_loss: 0.0018
Epoch 9/2000
42/42 [==============================] - 14s 323ms/step - loss: 3.9635e-04 - val_loss: 0.0014
Epoch 10/2000
42/42 [============================

INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_1000_batch_256_earlystop_valloss_4\assets


INFO:tensorflow:Assets written to: C:/Users/guilh/PythonCodes/Models/LSTM_predicting10ahead_param_14_ts_1000_batch_256_earlystop_valloss_4\assets
